In [22]:
%reload_kedro
train_data = catalog.load("train_data")
test_data = catalog.load("eval_data")
model = catalog.load("params:models.exponential_smoothing")
best_estimators = catalog.load("best_estimators")

2022-02-04 16:23:22,780 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.7.6
2022-02-04 16:23:22,874 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-02-04 16:23:22,928 - root - INFO - ** Kedro project time_series_kedro
2022-02-04 16:23:22,929 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-02-04 16:23:22,950 - root - INFO - Registered line magic `run_viz`
2022-02-04 16:23:22,951 - root - INFO - Registered line magic `reload_kedro_mlflow`
2022-02-04 16:23:22,952 - kedro.io.data_catalog - INFO - Loading data from `train_data` (CSVDataSet)...
2022-02-04 16:23:22,989 - kedro.io.data_catalog - INFO - Loading data from `eval_data` (CSVDataSet)...
2022-02-04 16:23:22,994 - kedro.io.data_catalog - INFO - Loading data from `params:models.exponential_smoothing` (MemoryDataSet)...
2022-02-04 16:23:22,995 - kedro.io.data_catalog - INF

In [23]:
train["family"].values[0]

'AUTOMOTIVE'

In [24]:
serie_id = "family"
serie_target = "sales"
date_col = "date"
train_data = pd.merge(train_data, best_estimators, on=serie_id)
group = train_data.group.values[0]
idx = train_data[serie_id].values[0]
y_true = test_data[test_data[serie_id] == idx].set_index(date_col)[serie_target]
ts =train_data.set_index(date_col)[serie_target]

In [25]:
y_true

date
2017-07-17    334.0
2017-07-18    301.0
2017-07-19    310.0
2017-07-20    325.0
2017-07-21    353.0
2017-07-22    493.0
2017-07-23    489.0
2017-07-24    347.0
2017-07-25    319.0
2017-07-26    296.0
2017-07-27    339.0
2017-07-28    344.0
2017-07-29    541.0
2017-07-30    529.0
2017-07-31    361.0
2017-08-01    454.0
2017-08-02    420.0
2017-08-03    368.0
2017-08-04    419.0
2017-08-05    513.0
2017-08-06    583.0
2017-08-07    355.0
2017-08-08    327.0
2017-08-09    314.0
2017-08-10    313.0
2017-08-11    441.0
2017-08-12    403.0
2017-08-13    481.0
2017-08-14    292.0
2017-08-15    337.0
Name: sales, dtype: float64

In [2]:
from time_series_kedro.pipelines.training.nodes import train_model

/home/matheus/.local/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


In [3]:
out = train_model(
    train,
    serie_id="family",
    serie_target="sales",
    date_col="date",
    model=model,
    stride=365,
    fr_horizon=7,
    initial=0.5,
    n_jobs=2)

In [4]:
out

,estimator,metric
family,,
AUTOMOTIVE,"ExponentialSmoothing(seasonal='add', seasonal_...",7.219484e+05
BABY CARE,"ExponentialSmoothing(seasonal='add', seasonal_...",1.501293e+06
BEAUTY,"ExponentialSmoothing(seasonal='add', seasonal_...",1.038293e+06
BEVERAGES,"ExponentialSmoothing(seasonal='add', seasonal_...",2.269157e+10
BOOKS,"ExponentialSmoothing(seasonal='add', seasonal_...",1.509583e+06
BREAD/BAKERY,"ExponentialSmoothing(seasonal='add', seasonal_...",7.146021e+08
CELEBRATION,"ExponentialSmoothing(seasonal='add', seasonal_...",4.410667e+05
CLEANING,ExponentialSmoothing(seasonal_periods=12),3.798677e+09
DAIRY,ExponentialSmoothing(seasonal_periods=12),2.044864e+09


In [5]:
model = catalog.load("params:models.svr")
out2 = train_model(
    train,
    serie_id="family",
    serie_target="sales",
    date_col="date",
    model=model,
    stride=365,
    fr_horizon=7,
    initial=0.5,
    n_jobs=2)

2022-02-04 15:39:11,495 - kedro.io.data_catalog - INFO - Loading data from `params:models.svr` (MemoryDataSet)...


In [6]:
out2

,estimator,metric
family,,
AUTOMOTIVE,"SVRForecaster(lags=3, poly_degree=1)",4.612267e+05
BABY CARE,"SVRForecaster(lags=12, poly_degree=1)",1.502679e+06
BEAUTY,"SVRForecaster(lags=3, poly_degree=1)",1.205447e+06
BEVERAGES,"SVRForecaster(lags=12, poly_degree=1)",3.362351e+10
BOOKS,"SVRForecaster(lags=6, poly_degree=1)",1.508850e+06
BREAD/BAKERY,"SVRForecaster(lags=12, poly_degree=1)",7.469282e+08
CELEBRATION,"SVRForecaster(lags=12, poly_degree=1)",7.834973e+05
CLEANING,"SVRForecaster(lags=3, poly_degree=1)",2.906357e+09
DAIRY,"SVRForecaster(lags=6, poly_degree=1)",1.446104e+09


In [8]:
import pandas as pd
estimators = pd.concat([out, out2]).reset_index().groupby("family").apply(lambda data: data.set_index("estimator").metric.idxmin())
estimators.name = "best_estimator"
estimators = estimators.reset_index()
estimators

,family,best_estimator
0,AUTOMOTIVE,"SVRForecaster(lags=3, poly_degree=1)"
1,BABY CARE,"ExponentialSmoothing(seasonal='add', seasonal_..."
2,BEAUTY,"ExponentialSmoothing(seasonal='add', seasonal_..."
3,BEVERAGES,"ExponentialSmoothing(seasonal='add', seasonal_..."
4,BOOKS,"SVRForecaster(lags=6, poly_degree=1)"
5,BREAD/BAKERY,"ExponentialSmoothing(seasonal='add', seasonal_..."
6,CELEBRATION,"ExponentialSmoothing(seasonal='add', seasonal_..."
7,CLEANING,"SVRForecaster(lags=3, poly_degree=1)"
8,DAIRY,"SVRForecaster(lags=6, poly_degree=1)"
9,DELI,"SVRForecaster(lags=6, poly_degree=1)"


/home/matheus/kedro_projects/time_series_kedro/src/time_series_kedro/pipelines/training/nodes/_train_model.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/matheus/.local/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
/home/matheus/kedro_projects/time_series_kedro/src/time_series_kedro/pipelines/training/nodes/_train_model.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/matheus/.local/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will

In [10]:
from sklearn.model_selection import GridSearchCV
from pmdarima.model_selection import RollingForecastCV
from time_series_kedro.pipelines.training.nodes._search import TSGridSearchCV
from time_series_kedro.extras.models import RandomForestForecaster
import pandas as pd

In [16]:
estimator = RandomForestForecaster(lags=12)
ts = train[train.family == train.family[0]].set_index("date").sales
param_grid = {"n_estimators": [10, 20, 30]}
cv = RollingForecastCV(step=365, h=30)
search = TSGridSearchCV(estimator, param_grid, cv_split=cv)
out = search.fit(ts)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   4 out of   9 | elapsed:    1.2s remaining:    1.5s
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:    2.1s finished
/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index i

In [13]:
out

,fold,estimator,training_status,error_message,metric
0,0,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,70949.562914
1,1,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,374700.389867
2,2,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,806716.229283
3,0,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,74148.762308
4,1,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,362803.751250
5,2,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,798174.388024
6,0,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,71831.831276
7,1,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,348190.557523
8,2,"RandomForestForecaster(lags=12, poly_degree=1)",success,None,800634.297412


In [15]:
def ld2dl(LD):
    return {k: [dic[k] for dic in LD] for k in LD[0]}
#out = pd.DataFrame(ld2dl(out))
out["estimator"] = out.estimator.apply(str)
out.groupby("estimator").mean().idxmin()["metric"]

'RandomForestForecaster(lags=12, poly_degree=1)'

In [9]:

pd.concat(map(pd.DataFrame, out), ignore_index=True)

ValueError: If using all scalar values, you must pass an index

In [ ]:
def ts_cv_split(
    ts: Union[pd.Series, np.array], 
    method: str = 'Rolling', 
    stride: int = 1, 
    frst: int =12, 
    pct_train: float = 0.7, 
    window_size: int = 24) -> Union[model_selection.RollingForecastCV, model_selection.SlidingWindowForecastCV]:

    
    if method == 'Rolling':
        
        initial = int(len(ts)*pct_train)
        n_trainings = len(ts) - (initial + frst)

        while (n_trainings < 3) & (frst != 1):
            frst -= 1
            n_trainings = len(ts) - (initial + frst)
            
        cv = model_selection.RollingForecastCV(step=stride, h=frst, initial=initial)
    
    elif method == 'SlidingWindow':
        cv = model_selection.SlidingWindowForecastCV(window_size=window_size, step=stride, h=frst)
        
    else: 
        raise NotImplementedError("The method you are trying to use is not implemented.")
    #logger.info(f"CV -> ts_size:{ts.shape[0]} initial:{initial} n_trainings:{n_trainings}")
    return cv

In [67]:
ts = train[train.family == train.family[0]].set_index("date").sales
model.fit(ts)
model.predict(12)

(1658, 13)
(1646, 12)


/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([498.89889577, 348.34279797, 263.89981864, 260.97329327,
       275.60332837, 311.42060238, 427.2548127 , 470.65903338,
       353.96486698, 242.67743698, 221.165882  , 233.42478792])

In [64]:
model.fit(ts)
model.predict(12)

(1658, 13)
(1646, 12)


/home/matheus/kedro_projects/time_series_kedro/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([396.6663898 , 233.80268208, 174.46021891, 179.49602484,
       172.38989678, 198.61140083, 292.54027608, 255.68566003,
       125.88984773,  70.25013873,  76.12819303,  65.66434245])

ValueError: Expected 2D array, got 1D array instead:
array=[393. 317. 317. 308. 517. 534. 321. 302. 286. 331. 325. 460.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.